# AIML Assignment 4
###   Gaofeng Shi
## Part 2

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import pandas as pd
import random
import seaborn as sns 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
bltin_sum = sum
from numpy import *
import warnings
warnings.filterwarnings('ignore')

## 2.2 Part 2: Performance Metrics in Classification 

from google.colab import drive
drive.mount('/content/drive')

In [2]:
# load the adult_data data
adult_data = pd.read_csv('adult.data', sep=',\s', skiprows=1, header=None, na_values=["?"])
adult_data.columns = ['A','B','C','D','E','F','G','H','I','K','L','M','N','O','class']
print("The number of missing values in the data before imputation :\n", adult_data.isnull().sum())
adult_data.head()

The number of missing values in the data before imputation :
 A           0
B        1836
C           0
D           0
E           0
F           0
G        1843
H           0
I           0
K           0
L           0
M           0
N           0
O         583
class       0
dtype: int64


,A,B,C,D,E,F,G,H,I,K,L,M,N,O,class
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


In [3]:
# load the adult_test data
adult_test = pd.read_csv('adult.test', sep=',\s', skiprows=1, header=None, na_values=["?"])
adult_test.columns = ['A','B','C','D','E','F','G','H','I','K','L','M','N','O','class']
print("The number of missing values in the data before imputation :\n", adult_test.isnull().sum())
adult_test.head()

The number of missing values in the data before imputation :
 A          0
B        963
C          0
D          0
E          0
F          0
G        966
H          0
I          0
K          0
L          0
M          0
N          0
O        274
class      0
dtype: int64


,A,B,C,D,E,F,G,H,I,K,L,M,N,O,class
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K.


In [4]:
# encoding the nomical features to numerical 
from sklearn.preprocessing import OrdinalEncoder 
encoder = OrdinalEncoder()
adult_data.loc[adult_data.B.notna(), ['B']] = encoder.fit_transform(adult_data["B"].dropna().values.reshape(-1,1))
adult_test.loc[adult_test.B.notna(), ['B']] = encoder.transform(adult_test["B"].dropna().values.reshape(-1,1))
adult_data.D = encoder.fit_transform(adult_data.D.values.reshape(-1, 1))
adult_test.D = encoder.transform(adult_test.D.values.reshape(-1, 1))
adult_data.F = encoder.fit_transform(adult_data.F.values.reshape(-1, 1))
adult_test.F = encoder.transform(adult_test.F.values.reshape(-1, 1))
adult_data.loc[adult_data.G.notna(), ['G']] = encoder.fit_transform(adult_data["G"].dropna().values.reshape(-1,1))
adult_test.loc[adult_test.G.notna(), ['G']] = encoder.transform(adult_test["G"].dropna().values.reshape(-1,1))
adult_data.H = encoder.fit_transform(adult_data.H.values.reshape(-1, 1))
adult_test.H = encoder.transform(adult_test.H.values.reshape(-1, 1))
adult_data.I = encoder.fit_transform(adult_data.I.values.reshape(-1, 1))
adult_test.I = encoder.transform(adult_test.I.values.reshape(-1, 1))
adult_data.K = encoder.fit_transform(adult_data.K.values.reshape(-1, 1))
adult_test.K = encoder.transform(adult_test.K.values.reshape(-1, 1))
adult_data.loc[adult_data.O.notna(), ['O']] = encoder.fit_transform(adult_data["O"].dropna().values.reshape(-1,1))
adult_test.loc[adult_test.O.notna(), ['O']] = encoder.transform(adult_test["O"].dropna().values.reshape(-1,1))
adult_data['class'] = adult_data['class'].map({'>50K': 1, '<=50K': 0})
adult_test['class'] = adult_test['class'].map({'>50K.': 1, '<=50K.': 0})
#adult_data.to_csv('adult_data.csv',index=False)
#adult_test.to_csv('adult_test.csv',index=False)

In [5]:
# get the lable class data
def data_process(dataset):
    X = dataset.iloc[:, 0:(len(dataset.columns)-1)]
    X = pd.DataFrame(StandardScaler().fit_transform(X))
    y = dataset.iloc[:, -1]
    data=(X,y)
    return data

In [6]:
# set the data for training and classify
train_X, train_y = data_process(adult_data)
test_X, test_y = data_process(adult_test)

In [7]:
# Imputer the data missing value with MICE
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.tree import DecisionTreeRegressor
#print("The number of missing values in the train data before imputation :\n", train_X.isnull().sum())
imputer_mice = IterativeImputer(max_iter=5, estimator
                                =DecisionTreeRegressor(),random_state=18)
train_X_mice = imputer_mice.fit_transform(train_X)
test_X_mice = imputer_mice.transform(test_X)
#The output is 'numpy.ndarray' so we convert it to dataframe for consistency
train_X_mice=pd.DataFrame(train_X_mice, columns=train_X.columns)
test_X_mice=pd.DataFrame(test_X_mice, columns=train_X.columns)
#print("The number of missing values :\n", train_X_mice.isnull().sum())

In [8]:
# Apply ten classifiy methods get the results.
from sklearn.datasets import make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn import metrics

acc, prec, recal, f1scor, auc = np.zeros(10),np.zeros(10),np.zeros(10),np.zeros(10),np.zeros(10)
train_X = train_X_mice
test_X = test_X_mice
classifications = [KNeighborsClassifier(n_neighbors=3),
                   GaussianNB(),
                   SVC(),
                   DecisionTreeClassifier(),
                   RandomForestClassifier(),
                   AdaBoostClassifier(),
                   GradientBoostingClassifier(),
                   LinearDiscriminantAnalysis(),
                   MLPClassifier(),
                   LogisticRegression()]
i = 0
for clsify in classifications:
    cls = clsify.fit(train_X, train_y)
    y_pred = cls.predict(test_X)
    acc[i] = accuracy_score(test_y, y_pred)
    prec[i] = precision_score(test_y, y_pred)
    recal[i] = recall_score(test_y, y_pred)
    f1scor[i] = f1_score(test_y, y_pred)
    auc[i] = roc_auc_score(test_y, y_pred)
    i += 1

In [9]:
# make the table of the results. 
ACC = np.around(acc,  decimals=2, out=None)
PRECISION = np.around(prec,  decimals=2, out=None)
RECALL = np.around(recal,  decimals=2, out=None)
F1_score = np.around(f1scor,  decimals=2, out=None)
AUC = np.around(auc,  decimals=2, out=None)
d1 = {'Classification': ['KNeighbors:', 'NaiveBayes:', 'SVM:', 'DecisionTree:', 'RandomForest:', 'AdaBoost:',
                    'GradientBoosting','LinearDiscriminantAnalysis:','Multi-Layer Perceptron:','Logistic Regression:'],
                    'Accuracy': ACC, 'Precision': PRECISION, 'Recall': RECALL, 'F1_score': F1_score, 'AUC': AUC}
results1 = pd.DataFrame(d1) #.set_index('Regression')

In [11]:
# output file sampleoutput.txt
import sys
f = open("sampleoutput.txt", 'w')
original_stdout = sys.stdout
sys.stdout = f
print("The result: ")
print(results1)
results1

,Classification,Accuracy,Precision,Recall,F1_score,AUC
0,KNeighbors:,0.82,0.63,0.57,0.60,0.74
1,NaiveBayes:,0.81,0.68,0.34,0.46,0.65
2,SVM:,0.85,0.75,0.55,0.63,0.75
3,DecisionTree:,0.81,0.59,0.60,0.60,0.74
4,RandomForest:,0.85,0.73,0.60,0.66,0.77
5,AdaBoost:,0.86,0.76,0.59,0.66,0.76
6,GradientBoosting,0.86,0.78,0.58,0.67,0.77
7,LinearDiscriminantAnalysis:,0.82,0.69,0.41,0.51,0.68
8,Multi-Layer Perceptron:,0.85,0.71,0.61,0.65,0.77
9,Logistic Regression:,0.83,0.70,0.45,0.55,0.70
